In [6]:
import os
import torch
import torch.nn as nn
import torch.distributed as dist
from torch.nn.parallel import DistributedDataParallel as DDP

from torch.distributed.fsdp import FullyShardedDataParallel as FSDP
from torch.distributed.fsdp import MixedPrecision


In [7]:
def setup_distributed():
    """
    如果在 torchrun 环境下（有 RANK/WORLD_SIZE），初始化 DDP；
    否则退化为单进程单卡模式，方便在 notebook 里调试。
    """
    if "RANK" in os.environ and "WORLD_SIZE" in os.environ:
        # 多进程模式：由 torchrun 设置好 env
        dist.init_process_group(backend="nccl", init_method="env://")
        local_rank = int(os.environ.get("LOCAL_RANK", 0))
        torch.cuda.set_device(local_rank)
        is_distributed = True
    else:
        # notebook / 单进程模式
        print("[setup_distributed] No RANK/WORLD_SIZE found, running in single-process mode.")
        local_rank = 0
        if torch.cuda.is_available():
            torch.cuda.set_device(local_rank)
        is_distributed = False

    return local_rank, is_distributed


def cleanup_distributed(is_distributed: bool):
    if is_distributed and dist.is_initialized():
        dist.destroy_process_group()


In [8]:
# Distributed Data Parallel

In [9]:
def run_ddp_demo():
    local_rank, is_distributed = setup_distributed()

    if is_distributed:
        rank = dist.get_rank()
    else:
        rank = 0

    device = torch.device("cuda", local_rank) if torch.cuda.is_available() else torch.device("cpu")

    model = nn.Linear(4096, 4096).to(device)

    if is_distributed:
        model = DDP(model, device_ids=[local_rank])
        print(f"[DDP] Running with world_size={dist.get_world_size()}, rank={rank}, local_rank={local_rank}")
    else:
        print("[DDP] Running in single-process, non-distributed mode.")

    x = torch.randn(32, 4096, device=device)
    loss = model(x).pow(2).mean()
    loss.backward()

    print(f"[DDP] rank {rank} finished one backward step, loss={loss.item():.4f}")

    cleanup_distributed(is_distributed)


In [10]:
run_ddp_demo()


[setup_distributed] No RANK/WORLD_SIZE found, running in single-process mode.
[DDP] Running in single-process, non-distributed mode.
[DDP] rank 0 finished one backward step, loss=0.3340


In [ ]:
# Fully Sharded Data Parallel

In [28]:
def run_fsdp_demo():
    local_rank = setup_distributed()
    rank = dist.get_rank()

    device = torch.device(f"cuda:{local_rank}")

    # 这里用一个简单的 Linear 模型做 demo
    model = nn.Linear(4096, 4096, device=device)

    mp = MixedPrecision(
        param_dtype=torch.bfloat16,
        reduce_dtype=torch.bfloat16,
        buffer_dtype=torch.bfloat16,
    )

    fsdp_model = FSDP(model, mixed_precision=mp, device_id=device)

    x = torch.randn(32, 4096, device=device)
    loss = fsdp_model(x).pow(2).mean()
    loss.backward()

    print(f"[FSDP] global rank {rank}, local_rank {local_rank}, loss = {loss.item():.4f}")

    cleanup_distributed()


/usr/local/lib/python3.12/dist-packages/torch/distributed/fsdp/_init_utils.py:430: UserWarning: FSDP is switching to use `NO_SHARD` instead of ShardingStrategy.FULL_SHARD since the world size is 1.
  warnings.warn(
